In [ ]:
# THINK THIS IS REDUNDANT CODE BUT CHECK CLASS AS MIGHT BE USEFUL

import yaml
from datetime import datetime, timedelta
import base64
import requests
from urllib.parse import quote
import os
from pathlib import Path
from ebaysdk.trading import Connection
from ebaysdk.exception import ConnectionError
from ebaysdk.finding import Connection

class EbayTokenManager:
    def __init__(self, config_file='ebay_config.yaml'):
        self.config_file = config_file
        self.config = self._load_config()

    def _load_config(self):
        """Load configuration from YAML file or create with defaults"""
        if os.path.exists(self.config_file):
            with open(self.config_file, 'r') as file:
                return yaml.safe_load(file)
        return {
            'credentials': {
                'client_id': None,
                'client_secret': None,
                'ru_name': None,
            },
            'tokens': {
                'refresh_token': None,
                'created_date': None,
                'access_token': None
            }
        }

    def _save_config(self):
        """Save configuration to YAML file"""
        with open(self.config_file, 'w') as file:
            yaml.dump(self.config, file)

    def setup_credentials(self, client_id, client_secret, ru_name):
        """Store eBay API credentials"""
        self.config['credentials']['client_id'] = client_id
        self.config['credentials']['client_secret'] = client_secret
        self.config['credentials']['ru_name'] = ru_name
        self._save_config()

    def store_refresh_token(self, refresh_token):
        """Store new refresh token with current date"""
        self.config['tokens']['refresh_token'] = refresh_token
        self.config['tokens']['created_date'] = datetime.now().isoformat()
        self._save_config()

    def should_refresh_token(self):
        """Check if refresh token is older than 1 year"""
        if not self.config['tokens']['created_date']:
            return True
            
        created_date = datetime.fromisoformat(self.config['tokens']['created_date'])
        one_year_old = datetime.now() - timedelta(days=365)
        return created_date < one_year_old

    def get_access_token(self):
        """Get new access token using refresh token"""
        if not self.config['tokens']['refresh_token']:
            raise Exception("No refresh token available. Please generate one first.")

        if not self.config['credentials']['client_id'] or not self.config['credentials']['client_secret'] or not self.config['credentials']['ru_name']:
            raise Exception("API credentials not set. Please run setup_credentials() first.")
            
        url = 'https://api.ebay.com/identity/v1/oauth/token'
        
        credentials = f"{self.config['credentials']['client_id']}:{self.config['credentials']['client_secret']}::{self.config['credentials']['ru_name']}"
        encoded_credentials = base64.b64encode(credentials.encode('utf-8')).decode('utf-8')
        
        headers = {
            'Content-Type': 'application/x-www-form-urlencoded',
            'Authorization': f'Basic {encoded_credentials}'
        }
        
        encoded_refresh_token = quote(self.config['tokens']['refresh_token'].strip())
        data = f"grant_type=refresh_token&refresh_token={encoded_refresh_token}"

        response = requests.post(url, headers=headers, data=data)

        if response.status_code == 200:
            access_token = response.json()['access_token']
            self.config['tokens']['access_token'] = access_token
            self._save_config()
            return access_token
        else:
            raise Exception(f"Failed to get access token. Status: {response.status_code}")

def main():

    manager = EbayTokenManager()

    # First time setup (only need to do this once):
    if not manager.config['credentials']['client_id']:
        manager.setup_credentials(
            client_id="TomArund-ReadInve-PRD-e66f79d08-8f8acc4d",
            client_secret="PRD-66f79d08e8ad-bf57-4662-affd-8697",
            ru_name = "Tom_Arundel-TomArund-ReadIn-zrmym"
        )
        manager.store_refresh_token('YOUR_INITIAL_REFRESH_TOKEN')

    # Regular usage
    try:
        if manager.should_refresh_token():
            print("WARNING: Refresh token is older than 1 year or doesn't exist!")
            print("Please generate a new refresh token and use store_refresh_token()")

        access_token = manager.get_access_token()
        print("Successfully obtained access token")
        return access_token

    except Exception as e:
        print(f"Error: {e}")
        return None

if __name__ == "__main__":
    main()


"""
The YAML file (ebay_config.yaml) will now look like this:

yamlCopycredentials:
  client_id: 'YOUR_CLIENT_ID'
  client_secret: 'YOUR_CLIENT_SECRET'
tokens:
  refresh_token: 'v^1.1#i^1#...'
  created_date: '2025-01-28T14:48:06.123456'
  access_token: 'v^1.1#i^1#...'

To use it:

First time setup:

pythonCopymanager = EbayTokenManager()
manager.setup_credentials(
    client_id='your_client_id',
    client_secret='your_client_secret'
)
manager.store_refresh_token('your_refresh_token')

Regular usage:

pythonCopymanager = EbayTokenManager()
access_token = manager.get_access_token()
The benefits of this approach:

All credentials and tokens in one place
Only need to set up credentials once
Clear separation between credentials and tokens
Easy to backup and version control (though make sure to .gitignore the yaml file!)
"""

In [5]:


headers = {
    "X-EBAY-API-SITEID" :3,
    "X-EBAY-API-COMPATIBILITY-LEVEL" :967,
    "X-EBAY-API-CALL-NAME" : "GetMyeBaySelling", 
    "X-EBAY-API-IAF-TOKEN": "v^1.1#i^1#r^0#p^3#f^0#I^3#t^H4sIAAAAAAAAAOVafWwbZxmP89HRdlk3wUZVVZPnUXUjOvu989fdEXtz4kviJE5c22nTSJv13t17ziX34d69l8ThQ1mGhjppU0UFSN0fpDCBhpA2Om1oYqvYVD6EtMGQitD+AMTH6KAgBpSPqtJ4z05TN9AmscM4CStSfO89X7/nfZ7nfe45g6UdOz/62NBjf+/23dK+sgSW2n0+ejfYuaOr57aO9n1dbaCBwLey9JGlzuWOC7021LUKn0d2xTRs5F/QNcPma4uJgGMZvAlt1eYNqCObxxJfSGVHeSYI+IplYlMytYA/k04EZElhOSnGIBEyIheWyapxVWbRTAQURIejYRpxCku+KOS2bTsoY9gYGjgRYAATpQBNMWyRpnk6ztNMkAXMVMB/GFm2ahqEJAgCyZq1fI3XajD15pZC20YWJkICyUxqoDCeyqSFsWJvqEFWctUNBQyxY19/1W/KyH8Yag66uRq7Rs0XHElCth0IJesarhfKp64a04T5NU9HGBkwEIoRBcVkuE2uHDAtHeKb2+GuqDKl1Eh5ZGAVVzfyKPGGOIMkvHo1RkRk0n733yEHaqqiIisREPpSRycKQj7gL+Ryljmnykh2kdJcLE7+WCaQxKauV0vYNDUdGqt66sJWvbxOUb9pyKrrM9s/ZuI+RIxG610DGlxDiMaNcSulYNegRrrwqguJJVPuntY30cHThrutSCd+8NcuN96AqxFxLQa2KyZQJMpFJS4mioiJIlH+DzHh5vqW4yLpbk0qlwu5tpDUrlI6tGYRrmhQQpRE3OvoyFJlnqQ0E2YVRMkxTqEinKJQYlSOUbSCEEBIFCWO/T8KD4wtVXQwWguR9TdqGBOBgmRWUM7UVKkaWE9SqzirAbFgJwLTGFf4UGh+fj44Hw6aVjnEAECHJrOjBWka6TCwRqtuTEyptdCQEOGyVR5XK8SaBRJ5RLlRDiTDlpyDFq4WkKaRhatxe51tyfWrNwDZr6nEA0WiwlsYh0wbI7klaDKaUyVUUmVvIXNznWHCcZZmQQQAwLYEUjPLqpFFeNr0GEy3KGTSLWEjNRRib6FqqC4gslqF4gxHkW8AtAQWli1Uq0b1hsRbsFP9/UKuKLS2nalKJaPrDoaihjIei9Yo6Xtj4e3ZQGG1+SJkbq57BqN7VJf6x7NZId8vlCZGSqPF1ja04jheK64T6VSfIFgwz+gtQXPbKV6FCo/NWWR473jMCwN5oTBUKo6PCGMtIc0jxUL2dNHF6bWsTB1KjaTIJyv0QNPsG8nbjANGJ2AeO4cnI3NSenJxYXFqkc4p5TklvMhFJueyBTQ1Y8QksWcsp7FCZjqLh2er84lES04qIMlCGxxFbq6/3w6a7ekZnJzLlefFYWYgfuiYFnV0cZKNHo2O55W0rYf6JxaGY8wRR5pvzQHZstcyfftaqKI3U9yqJ2apVoFK5KolkELZc7XaHUlFEBOjuQiAMSCFRRaJSGEVReEYEdAtNxv/PbxurjeFuWjqKcsxZCqPoJwx5hCVy6cpFIspcU4GLMUqLJSkSGsPPhXPbfV2HctrPZZ76S2IudTRrDBWLDAlUHKffUqpwbwgZK9N3ppDbLtDCG8hdfltIgBW1KDbJwUlUw+Z0MHT7lKpZrF/M0QhG2lasD60IpKDFskJ09CqzTBvgUclaWdg06puTqGb6zcQsAWlJKtNx8DNYFxl3QKH4miKqmluojSjsIF9K2YaUKtiVbKbUqkabsTZW2CpwGoNoKzaFTdfNsVJ1nRkSSioyvUZeTPGWogohLWRcDNMW1S5ZrJhYlVRpboM2xFtyVIrm7diYznN+MImebClbaszbEpVAxeSkabOIava2kTQ1FVJ1W5cT91c/5/U1MG+1iZlSFYtJOGSY6neOixIw1OqdTxIo9Y1P9SipVdb7AdImHhx+JlLFQpHxvOtjVfSaM5rD+QcghGRi4QpwDA0FQlHEcWykKNEBkYRC8MAxFt7FnsfB76dj5zcHGg6HmZBLAzCmx7wrltoeNH0b68YQ9e/4U+21T70su81sOw72+7zgV5wgL4X3LOjY6Kz49Z9torJ4QWVoK2WDYgdCwVnUbUCVav9g20/vG1UfmRo9NKS6HzzyF8fYNu6G35gsPIQ2Lv2E4OdHfTuht8bgP3X7nTRez7czUQBzbA0TcdpZgrce+1uJ31X54fOn/nWWxd2/+3FrmetttuPf3n2vT/sfxd0rxH5fF1tncu+tk+9efc7P7hl4MQ78c/85eeXz8R+fecn7nrm5fPUE5+PCa9H73jy3cLAiU9+9o37p2eqz9ts7xvHOi+ffuXAPy5/ffClL7UfGPrq13pKPStTnz7355Bx/2DygbO5RM+p35vD3xBO7T1x4vHOhx8dea735I9++s9bxe917Tr3q+fSM5e4t5/15/Clp5bA/pGvXHzod9Gn+34zv+OLu648Pay/1z71syuvstKjpy5+J7mnOyI+uGvlvmeeP9/x4Nu+8+bpjxXfevH21+YvXui7/NsrmadeOLP3u0/cfc/w0Me/XXkdHvzx0V0HF6F1ah/3y3z4zrPLXziYGpeOdf/p+12f+8meh5kP7H+849wl8aWTx+87/sfTr87Qk794k6vv5b8AJjStGPohAAA="
}

ebay_cwd = "/Users/wommy/Documents/GitHub/PROJECTS/HANKS/inventory_system/app/services/ebay/ebaysdk-python"

# print(os.getcwd())
if os.getcwd() != ebay_cwd:
    os.chdir(ebay_cwd)

try:
    api = Connection(appid='TomArund-ReadInve-PRD-e66f79d08-8f8acc4d', config_file='ebay.yaml')
    response = api.execute('GetStore', {'keywords': 'iphone'})

    assert(response.reply.ack == 'Success')
    assert(type(response.reply.timestamp) == datetime.datetime)
    assert(type(response.reply.searchResult.item) == list)

    item = response.reply.searchResult.item[0]
    assert(type(item.listingInfo.endTime) == datetime.datetime)
    assert(type(response.dict()) == dict)

except ConnectionError as e:
    print(e)
    print(e.response.dict())


'GetStore: Internal Server Error, Domain: CoreRuntime, Severity: Error, errorId: 2000, Service operation GetStore is unknown'
{'errorMessage': {'error': {'errorId': '2000', 'domain': 'CoreRuntime', 'severity': 'Error', 'category': 'Request', 'message': 'Service operation GetStore is unknown', 'subdomain': 'Inbound_Meta_Data', 'parameter': {'_name': 'Param1', 'value': 'GetStore'}}}}


In [6]:

def get_oauth_token(client_id, client_secret):
    url = "https://api.ebay.com/identity/v1/oauth/token"

    # Encode credentials
    credentials = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Authorization": f"Basic {credentials}"
    }

    data = {
        "grant_type": "client_credentials",
        "scope": "https://api.ebay.com/oauth/api_scope"  # Add more scopes as needed
    }

    response = requests.post(url, headers=headers, data=data)
    print(response)
    return response.json()["access_token"]

token = get_oauth_token(client_id, client_secret)

NameError: name 'client_id' is not defined